In [ ]:
import pandas as pd
import warnings
import numpy as np
import sklearn

warnings.filterwarnings("ignore")

In [ ]:
tt_filepath = "./output/train_test.csv"
v_filepath = "./output/validation.csv"
train_test = pd.read_csv(tt_filepath)
vali = pd.read_csv(v_filepath)


In [ ]:


data_t =  pd.concat([train_test[["label"]],train_test.loc[:,'SEX':"PET_original_gldm_SmallDependenceLowGrayLevelEmphasis"]],axis=1)
data_v = pd.concat([vali[["label"]],vali.loc[:,'SEX':"PET_original_gldm_SmallDependenceLowGrayLevelEmphasis"]],axis=1)

In [ ]:
data_t = sklearn.utils.shuffle(data_t)
data_v = sklearn.utils.shuffle(data_v)

X = data_t.loc[:,"SEX":]
y = data_t['label']
Xvv = data_v.loc[:,"SEX":]
y_v = data_v["label"]
columns = Xvv.columns
X.head(5)

In [ ]:
from sklearn.linear_model import LassoCV
alphas = np.logspace(-3,1,100)
model_lassoCV = sklearn.linear_model.LassoCV(alphas = alphas, cv = 10, max_iter = 1000000).fit(X,y)
print("alpha = " + str(model_lassoCV.alpha_))
coef = pd.Series(model_lassoCV.coef_,index = X.columns)
print('Lasso picked ' + str(sum(coef !=0))+' variables and eliminated the other ' + str(sum(coef == 0)))

index = coef[coef != 0].index
X_t = X[index]
print(coef[coef !=0])


In [ ]:
from sklearn.linear_model import Lasso
model_lasso = Lasso(alpha=0.045, max_iter=10000).fit(X,y)
coef_ = pd.Series(model_lasso.coef_,index = X.columns)
print('Lasso picked ' + str(sum(coef_ !=0))+' variables and eliminated the other ' + str(sum(coef_ == 0)))

index = coef_[coef_ != 0].index
X_t = X[index]
print(coef_[coef_ !=0])

In [ ]:
import itertools
comb = list(itertools.combinations(index,5))
print(len(comb))
# print(comb)

In [ ]:
from model import cross_model
X = pd.DataFrame()
X_v = pd.DataFrame()
score = pd.DataFrame()
col = []
a = 0
for non in comb:
    cli_ = [cli_ for cli_ in non if cli_  in cli]
    ct_ = [ct_ for ct_ in non if ct_ in ct]
    pet_ = [pet_ for pet_ in non if pet_ in pet]
    if len(cli_) != 0 and len(ct_) != 0 and len(pet_) != 0:
        col.append(non)
        for i in non:
            X_v = pd.concat([X_v,Xvv[[i]]],axis=1)
            X = pd.concat([X,X_t[[i]]],axis=1)
        score_rf, score_rf_v,__,__ = cross_model.rf(X, y, X_v, y_v)
        score_NB, score_NB_v,__,__ = cross_model.NB(X, y, X_v, y_v)
        score_Gtree, score_Gtree_v,__,__ = cross_model.Gtree(X, y, X_v, y_v)
        score_gbm, score_gbm_v,__,__ = cross_model.gbm(X, y, X_v, y_v)
        score_xgb, score_xgb_v,__,__ = cross_model.xgb(X, y, X_v, y_v)
        score_lore, score_lore_v,__,__ = cross_model.lore(X, y,X_v, y_v)
        score1 = [[score_rf, score_rf_v,score_NB, score_NB_v,score_Gtree, score_Gtree_v,
                score_gbm, score_gbm_v,score_xgb, score_xgb_v,score_lore, score_lore_v]]
        score2 = pd.DataFrame(score1)
        score = pd.concat([score,score2],axis=0)
        X = pd.DataFrame()
        X_v = pd.DataFrame()
        a = a + 1
    if a % 1000 == 0:
        score.to_csv("./output/test1.csv",sep=',',mode="a",index=False,header=None)
        score = pd.DataFrame()


In [ ]:
print(len(col))
col2 = col[0:27000]
print(len(col2))

In [ ]:
test_path = "./output/test1.csv"
test = pd.read_csv(test_path)
test.iloc[[0]]

In [ ]:
def stat(feature):
    c = []
    data_f = pd.DataFrame()
    des = pd.DataFrame()
    for b in col2:
        if feature in b:
            c1 = col2.index(b)
            c.append(c1)
    for c2 in c:
        f1 = test.iloc[[c2]]
        data_f = pd.concat([data_f,f1],axis=0)
    for fea in data_f.columns:
        des1 = data_f[[fea]].describe()
        des2 = pd.DataFrame(des1)
        des2.columns = [feature]
        des = pd.concat([des,des2],axis=0)
    return des
    


In [ ]:
feature_select = pd.DataFrame()
for feature in index:
    feature_s = stat(feature)
    feature_select = pd.concat([feature_select,feature_s],axis=1)
feature_select.to_csv("./output/feature_select.csv")

In [ ]:
feature_select = pd.read_csv("./output/feature_select.csv")

In [ ]:
from statsmodels.api import OLS
import statsmodels.api as sm
def stepwise_selection(data, target,SL_in=0.05,SL_out = 0.05):
    initial_features = data.columns.tolist()
    best_features = []
    while (len(initial_features)>0):
        remaining_features = list(set(initial_features)-set(best_features))
        new_pval = pd.Series(index=remaining_features)
        for new_column in remaining_features:
            model = OLS(target, sm.add_constant(data[best_features+[new_column]])).fit()
            new_pval[new_column] = model.pvalues[new_column]
        min_p_value = new_pval.min()
        if(min_p_value<SL_in):
            best_features.append(new_pval.idxmin())
            while(len(best_features)>0):
                best_features_with_constant = sm.add_constant(data[best_features])
                p_values = OLS(target, best_features_with_constant).fit().pvalues[1:]
                max_p_value = p_values.max()
                if(max_p_value >= SL_out):
                    excluded_feature = p_values.idxmax()
                    best_features.remove(excluded_feature)
                else:
                    break 
        else:
            break
    return best_features

In [ ]:
stepwise_selection(X,y)

In [ ]:
from statsmodels.formula.api import ols

def forward_select(data,target):
    variate=list(set(data.columns))  
    variate.remove(target)  
    selected=[]
    selected_b = []
    current_score,best_new_score=float('inf'),float('inf') 
    current_score_b,best_new_score_b=float('inf'),float('inf')
    while variate:
        aic_with_variate=[]
        aic_with_variate_b=[] 
        for candidate in variate:  
            formula="{}~{}".format(target,"+".join(selected+[candidate])) 
            aic=ols(formula=formula,data=data).fit().aic  
            aic_with_variate.append((aic,candidate))
              
        aic_with_variate.sort(reverse=True)  
        best_new_score,best_candidate=aic_with_variate.pop() 
        if current_score>best_new_score and best_new_score > -100:  
            selected.append(best_candidate)  
            current_score=best_new_score  
            print("aic is {},continuing!".format(current_score))  
        
        else:
            break
    return selected
        
        
       
    


In [ ]:
data = pd.concat([X_t,y],axis=1)
data.head(5)

In [ ]:
forward_select(data,"label")

In [ ]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2

X1 = sklearn.preprocessing.MinMaxScaler().fit_transform(X)
model_chi = SelectKBest(chi2,k=5)
model_chi.fit_transform(X1,y)
model_chi.scores_
X.columns[model_chi.get_support(True)]

In [ ]:
import lightgbm as lgb
import matplotlib.pyplot as plt
scorel = []
for i in range(0,20,1):
    model = lgb.LGBMClassifier(n_estimators=i+1,
                                 n_jobs=--4,
                                 random_state=90).fit(X,y)
    score = model.score(X,y)
    scorel.append(score)

print(max(scorel),(scorel.index(max(scorel))*1)+1)  
plt.figure(figsize=[20,5])
plt.plot(range(0,20,1),scorel)
plt.savefig("./result/feature_selection/LightGBM_num.png",dpi=600,bbox_inches = 'tight')

plt.show()

In [ ]:
n = 15
model = lgb.LGBMClassifier().fit(X, y)
lgb.plot_importance(model,  max_num_features=n, figsize=(10,5),importance_type='split')

plt.savefig("./result/feature_selection/LightGBM_CT.png",dpi=600,bbox_inches = 'tight')
plt.show()

In [ ]:
feature_weight = model.feature_importances_
feature_name = model.feature_name_
feature_sort = pd.Series(data = feature_weight ,index = feature_name)
feature_sort = feature_sort.sort_values(ascending = False)
feature_sort.index[0:n]

In [ ]:
from sklearn.model_selection import GridSearchCV
import matplotlib.pyplot as plt
import xgboost as xgb
from xgboost import plot_tree
scorel = []
for i in range(0,20,1):
    model = xgb.XGBClassifier(n_estimators=i+1,
                                 n_jobs=--4,
                                 random_state=90).fit(X,y)
    score = model.score(X,y)
    scorel.append(score)

print(max(scorel),(scorel.index(max(scorel))*1)+1)  
plt.figure(figsize=[20,5])
plt.plot(range(0,20,1),scorel)
plt.show()

In [ ]:

n = 10
model = xgb.XGBClassifier(eval_metric=['logloss','auc','error']).fit(X, y)
xgb.plot_importance(model,  max_num_features=n, importance_type='weight')
plt.savefig("./result/feature_selection/XGBoost_CT.png",dpi=600,bbox_inches = 'tight')

plt.show()



In [ ]:
importance = model.get_booster().get_score()
tuples = [(k, importance[k]) for k in importance]
tuples = sorted(tuples, key=lambda x: x[1],reverse=True)
feature_names,scores = map(list,zip(*tuples))
feature_names[0:n]